In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from transformers import AutoTokenizer

model_name = "EMBEDDIA/crosloengual-bert"

In [6]:
from datasets import load_dataset

dataset = load_dataset("lenatr99/Slovene_SuperGLUE_BoolQ")

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

/opt/anaconda3/envs/orange4/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
def preprocess_data(examples):
    passage_inputs = [f"passage : {x} " for x in examples["passage"]]
    question_inputs = [f"question : {x} " for x in examples["question"]]
    inputs = [passage_input + question_input for passage_input, question_input in zip(passage_inputs, question_inputs)]

    model_inputs = tokenizer(inputs, max_length=400, truncation=True)

    model_inputs["labels"] = examples["label"]
    model_inputs["labels"] = [int(label) for label in model_inputs["labels"]]

    return model_inputs

In [9]:
tokenized_dataset = dataset.map(preprocess_data, remove_columns=['idx', 'passage', 'label', 'question'], batched=True)

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {"accuracy": acc, "f1": f1}

In [12]:
from transformers import AutoModelForSequenceClassification, set_seed

set_seed(42)

id2label = {0: "False", 1: "True"}
label2id = {"False": 0, "True": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)

model.config.use_cache = False

/opt/anaconda3/envs/orange4/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/crosloengual-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from peft import LoHaConfig, get_peft_model, TaskType

loha_alpha = 32
loha_rank_dropout = 0.1
loha_module_dropout = 0.0
loha_r = 16

peft_config = LoHaConfig(
    task_type=TaskType.SEQ_CLS,
    r=loha_r,
    alpha=loha_alpha,
    target_modules=['query', 'value'],
    base_model_name_or_path=model_name
)

In [14]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,181,186 || all params: 125,317,636 || trainable%: 0.9426


In [15]:
from transformers import TrainingArguments

new_model_name = "loha_fine_tuned_boolq_croslo"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [17]:
trainer.train()

  0%|          | 0/400 [00:00<?, ?it/s]

{'loss': 0.7131, 'grad_norm': 2.8468222618103027, 'learning_rate': 1.7500000000000002e-05, 'epoch': 4.17}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.6457347869873047, 'eval_accuracy': 0.7222222222222222, 'eval_f1': 0.7325102880658435, 'eval_runtime': 3.7388, 'eval_samples_per_second': 4.814, 'eval_steps_per_second': 0.802, 'epoch': 4.17}
{'loss': 0.676, 'grad_norm': 3.7239277362823486, 'learning_rate': 1.5000000000000002e-05, 'epoch': 8.33}


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.5909553170204163, 'eval_accuracy': 0.8333333333333334, 'eval_f1': 0.8243021346469622, 'eval_runtime': 0.6719, 'eval_samples_per_second': 26.789, 'eval_steps_per_second': 4.465, 'epoch': 29.17}
{'loss': 0.6639, 'grad_norm': 3.2821714878082275, 'learning_rate': 0.0, 'epoch': 33.33}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.5903695225715637, 'eval_accuracy': 0.8333333333333334, 'eval_f1': 0.8243021346469622, 'eval_runtime': 0.409, 'eval_samples_per_second': 44.009, 'eval_steps_per_second': 7.335, 'epoch': 33.33}
{'train_runtime': 367.0245, 'train_samples_per_second': 8.719, 'train_steps_per_second': 1.09, 'train_loss': 0.680639591217041, 'epoch': 33.33}


TrainOutput(global_step=400, training_loss=0.680639591217041, metrics={'train_runtime': 367.0245, 'train_samples_per_second': 8.719, 'train_steps_per_second': 1.09, 'total_flos': 395431124581920.0, 'train_loss': 0.680639591217041, 'epoch': 33.333333333333336})

In [18]:
trainer.evaluate()

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.5903695225715637,
 'eval_accuracy': 0.8333333333333334,
 'eval_f1': 0.8243021346469622,
 'eval_runtime': 0.6257,
 'eval_samples_per_second': 28.769,
 'eval_steps_per_second': 4.795,
 'epoch': 33.333333333333336}

In [19]:
trainer.push_to_hub(new_model_name)

/opt/anaconda3/envs/orange4/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/4.74M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lenatr99/loha_fine_tuned_boolq_croslo/commit/a8563b5e28c16a8dcfd2a238d9c00fad83a16e63', commit_message='loha_fine_tuned_boolq_croslo', commit_description='', oid='a8563b5e28c16a8dcfd2a238d9c00fad83a16e63', pr_url=None, pr_revision=None, pr_num=None)

In [20]:
from transformers import pipeline

adapter_name = "lenatr99/" + new_model_name

# Example
text="passage : Bankovec za 20 evrov – Zaenkrat obstaja le ena celotna serija evrskih bankovcev, vendar pa izhaja nova serija, ki bo podobna sedanji. Evropska centralna banka bo pravočasno naznanila, kdaj bodo bankovci iz prve serije izgubili status zakonitega plačilnega sredstva. question : Ali je bankovec za 20 evrov iz prve serije še vedno zakonito plačilno sredstvo?"

classifier = pipeline("sentiment-analysis", model=adapter_name)
classifier(text)

adapter_config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/crosloengual-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_1', 'score': 0.5788205862045288}]